# Structure FE code for 2D heat flow
The following pseudo code shows how a FE-script could be structured. 
// Jim


# 1. Preprocessing
Define input variables: material parameters, mesh parameters, etc
Construct (or import mesh) -> mesh

# 2. Processing
Assemble the system of equations for convection $(\mathbf{K} + \mathbf{K}_c)\,\mathbf{a} = \mathbf{f}_l + \mathbf{f}_b + \mathbf{f}_c$

## 2.1 Element contributions: $\mathbf{K}$ and $\mathbf{f}_l$  
These are assembeled **element-wise**

```py
for el in range(num_el):
    ...
    nodes = ...
    Ke, fe = flow2t_Ke_fe(...)
    element_dofs = ...
    assem(K, Ke, element_dofs)
    assem(f, fle, element_dofs)
```

## 2.2 Boundary contributions due to convection: $\mathbf{K}_c$ and $\mathbf{f}_c$
Convective contributions are only nonzero on the convective boundaries of the structure. Therefore, these are assembled **edge-wise**. If you have multiple convective boundaries, write multiple loops (or nested loops).

```py
for edge in range(num_edges_on_convective_boundary)
    ...
    edge_nodes = ...
    edge_dofs = ...
    Ke_c, fe_c = convection_Ke_fe(...)
    fe_c = ...
    assem(K, Ke_c, edge_dofs)
    assem(f, fe_c, edge_dofs)
```

## 2.3 Boundary contributions due to prescribed flux: $\mathbf{f}_b$

Flux terms on the free part of the boundary $\mathbf{f}_b^\text{free}$ are evalued similarly as the convective terms but do not give any contribution to the stiffness matrix and we get
```py
for edge in range(number_of_edges_with_given_flux)
    ...
    edge_dofs = ...
    fe_b = ...
    assem(f, fe_b, edge_dofs)
```
Note: This only contributes if the prescribed flux is non-zero

## 2.4 Essential boundary conditions
Define prescribed nodal temperatures = the value of the dof in the corresponding node
```py
bc_dofs = np.array([dof_1, dof_2, ..., dof_n])  
bc_vals = np.array([value_in_dof_1, value_in_dof_2, value_in_dof_n])
```

## 2.5 Solve the system of equations taking into account essential boundary conditions
```py
a, r = solve_eq(K, f, bc_dofs, bc_vals)
```
If no essential boundary conditions are present, then simply use 
```py
a = np.linalg.solve(K, f)
```
In this case we don't get any reactions so that vector is simply a zero vector.

# 3 Postprocessing
Given the solution vector $\mathbf{a}$ all other (secondary quantities can be computed). Note that all quantities are computed either over each element or over the edges/segments.

## 3.1 Heat flux vector
These are computed **element-wise** with one vector per element (store them in a matrix [N, 2])

```py
q = np.zeros((num_el, 2))
for el in range(num_el):
    ...
    el_nodes = ...
    ae = ...
    qe = flow2t_qe(...)

```

## 3.2 Heat flow on a convective boundary
This it the amount of energy that enters or leaves a boundary and is computed **edge-wise** using the formula for heat flux $q_n = \alpha(T-T_\infty) = \alpha(\mathbf{N}^e \mathbf{a}^e-T_\infty) \Rightarrow$ Heat flow
$$
Q^{e}=\int_{\mathcal{L}^e} q_n t \mathrm{~d} \mathcal{L} = 
\int_{\mathcal{L}^e} \alpha_e\left(\mathbf{N}^e   \mathbf{a}^e-T_\infty\right) t \mathrm{~d} \mathcal{L}=\alpha_e L_e t\left(\frac{T_i^e+T_j^e}{2}-T_\infty\right)
$$
with 
$$
\mathbf{N}^{\mathrm{e}}=\left[\begin{array}{ll}
N_i^e & N_j^e
\end{array}\right] \text { and } \mathbf{a}^e=\left[\begin{array}{c}
T_i^e \\
T_j^e
\end{array}\right]
$$
The heat flow is the sum of all contributions $Q = \sum Q^e$ 
```py
Q = 0.0
for edge in range(num_edges_on_convective_boundary)
    ...
    edge_dofs = ...
    
    ae = ...
    Q += ... 

```